### Initialize Spark session

In [28]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

println("Initializing Spark context...")
val conf = new SparkConf().setAppName("Example App")
val spark: SparkSession = SparkSession.builder.config(conf).getOrCreate()
//if you look in Spark Master UI, a application will be running after this

Initializing Spark context...


conf = org.apache.spark.SparkConf@2bbffd76
spark = org.apache.spark.sql.SparkSession@1dcacea6


lastException: Throwable = null


org.apache.spark.sql.SparkSession@1dcacea6

### Custom Jar import

In [29]:
println("Import custom jar in the notebook with a special Toree directive")
%AddJar file:///app/app.jar

Import custom jar in the notebook with a special Toree directive
Starting download from file:///app/app.jar
Finished download of app.jar


In [30]:
println("Importing custom class")
import app.Point
val p = new Point(1,2)
p.move(3,5)

Importing custom class
Point x location : 4
Point y location : 7


p = app.Point@6db4b5e6


app.Point@6db4b5e6

### Hello World using remote Spark master

In [32]:
println("************")
println("Hello, world!")
val rdd = spark.sparkContext.parallelize(Array(1 to 10))
rdd.count()
println("************")

************
Hello, world!
************


rdd = ParallelCollectionRDD[22] at parallelize at <console>:136


ParallelCollectionRDD[22] at parallelize at <console>:136

### HDFS tests

In [33]:
val hdfsPrefix = sys.env("HDFS_URL")
val hadoopConf = new Configuration()
hadoopConf.set("fs.defaultFS", sys.env("HDFS_URL"))
val hdfs = FileSystem.get(hadoopConf)

hdfsPrefix = hdfs://namenode1:8020
hadoopConf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
hdfs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-915978460_230, ugi=root (auth:SIMPLE)]]


DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-915978460_230, ugi=root (auth:SIMPLE)]]

In [34]:
println("Copying sample file to HDFS...")
val srcPath = new Path("/notebooks/people.csv")
val destPath = new Path("hdfs:///test/people.csv")
hdfs.copyFromLocalFile(srcPath, destPath)

Copying sample file to HDFS...


srcPath = /notebooks/people.csv
destPath = hdfs:/test/people.csv


hdfs:/test/people.csv

In [35]:
println("Load CSV from HDFS to Dataframe")
val df = spark.read
          .format("com.databricks.spark.csv")
          .option("inferSchema", "true")
          .option("header", "true")
          .load(hdfsPrefix + "/test/people.csv")
df.describe().show()

Load CSV from HDFS to Dataframe
+-------+-----------------+------------------+------------------+
|summary|            Index|            Height|            Weight|
+-------+-----------------+------------------+------------------+
|  count|            25000|             25000|             25000|
|   mean|          12500.5| 67.99311359679979|127.07942116079916|
| stddev|7217.022700994273|1.9016787712056056|11.660897563604271|
|    min|                1|          60.27836|          78.01476|
|    max|            25000|           75.1528|           170.924|
+-------+-----------------+------------------+------------------+



df = [Index: int, Height: double ... 1 more field]


[Index: int, Height: double ... 1 more field]

In [36]:
println("Perform some SQL over CSV contents")
df.createOrReplaceTempView("people")
val df2 = spark.sql("SELECT * FROM people WHERE Height BETWEEN 68 AND 71")
df2.describe().show()

Perform some SQL over CSV contents
+-------+------------------+------------------+------------------+
|summary|             Index|            Height|            Weight|
+-------+------------------+------------------+------------------+
|  count|             11050|             11050|             11050|
|   mean|12598.144977375565| 69.22068299909508| 130.8976952787331|
| stddev| 7230.438697100867|0.8064637962888647|10.368627688958973|
|    min|                 3|          68.00003|          90.53995|
|    max|             25000|          70.99707|           170.924|
+-------+------------------+------------------+------------------+



df2 = [Index: int, Height: double ... 1 more field]


[Index: int, Height: double ... 1 more field]

In [37]:
println("Save CSV using Dataframe")
df2.repartition(5).write
   .format("com.databricks.spark.csv")
   .option("header", "true")
   .mode("overwrite")
   .save(hdfsPrefix + "/test/people-result.csv")

Save CSV using Dataframe


### Custom Maven dependency

In [26]:
print("Add Vegas lib")
%AddDeps org.vegas-viz vegas_2.11 0.3.11 --transitive
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.11

Add Vegas libMarking org.vegas-viz:vegas_2.11:0.3.11 for download
Obtained 42 files
Marking org.vegas-viz:vegas-spark_2.11:0.3.11 for download
Obtained 2 files


### Show some graphs (finally!)

In [5]:
import vegas._
import vegas.sparkExt._

Vegas("Person Heights", width=400, height=100)
  .withDataFrame(df)
  .encodeX("Index", Quantitative)
  .encodeY("Height", Quantitative)
  .mark(Bar)
  .show

Vegas("Person Weights", width=400, height=100)
  .withDataFrame(df)
  .encodeX("Index", Quantitative)
  .encodeY("Weight", Quantitative)
  .mark(Bar)
  .encodeColor(field="Cylinders", dataType=Nominal)
  .show

<iframe id="frame-vegas-7ca98aaf-237b-4c8a-b212-d764fba1c6b6" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-7ca98aaf-237b-4c8a-b212-d764fba1c6b6'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 400.0,
 "height" : 100.0,
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Index",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "Height",
 "type" : "quantitative"
 }
 },
 "description" : "Person Heights",
 "data" : {
 "values" : [
 {
 "Index" : 3,
 "Height" : 69.39874,
 "Weight" : 153.0269
 },
 {
 "Index" : 5,
 "Height" : 67.78781,
 "Weight" : 144.2971
 },
 {
 "Index" : 10,
 "Height" : 66.78236,
 "Weight" : 120.6672
 },
 {
 "Index" : 14,
 "Height" : 67.11656,
 "Weight" : 122.4618
 },
 {
 "Index" : 16,
 "Height" : 71.0916,
 "Weight" : 139.9975
 },
 {
 "Index" : 18,
 "Height" : 68.64927,
 "Weight" : 142.9733
 },
 {
 "Index" : 20,
 "Height" : 67.13118,
 "Weight" : 124.0449
 },
 {
 "Index" : 23,
 "Height" : 63.48115,
 "Weight" : 97.90191
 },
 {
 "Index" : 25,
 "Height" : 67.62804,
 "Weight" : 141.8501
 },
 {
 "Index" : 29,
 "Height" : 66.53401,
 "Weight" : 108.3324
 },
 {
 "Index" : 32,
 "Height" : 65.8132,
 "Weight" : 120.7536
 },
 {
 "Index" : 33,
 "Height" : 67.8163,
 "Weight" : 125.7886
 },
 {
 "Index" : 37,
 "Height" : 66.80368,
 "Weight" : 141.7994
 },
 {
 "Index" : 39,
 "Height" : 67.80701,
 "Weight" : 131.3478
 },
 {
 "Index" : 40,
 "Height" : 64.04535,
 "Weight" : 106.7115
 },
 {
 "Index" : 42,
 "Height" : 65.18357,
 "Weight" : 124.8591
 },
 {
 "Index" : 44,
 "Height" : 67.96731,
 "Weight" : 137.3696
 },
 {
 "Index" : 45,
 "Height" : 65.98088,
 "Weight" : 106.4499
 },
 {
 "Index" : 48,
 "Height" : 67.69868,
 "Weight" : 116.819
 },
 {
 "Index" : 49,
 "Height" : 69.82117,
 "Weight" : 143.6215
 },
 {
 "Index" : 57,
 "Height" : 70.40617,
 "Weight" : 155.8987
 },
 {
 "Index" : 59,
 "Height" : 66.36418,
 "Weight" : 119.3701
 },
 {
 "Index" : 61,
 "Height" : 66.50418,
 "Weight" : 128.7325
 },
 {
 "Index" : 66,
 "Height" : 68.21951,
 "Weight" : 109.6113
 },
 {
 "Index" : 67,
 "Height" : 69.05914,
 "Weight" : 142.4684
 },
 {
 "Index" : 68,
 "Height" : 67.73103,
 "Weight" : 132.749
 },
 {
 "Index" : 69,
 "Height" : 67.21568,
 "Weight" : 103.5275
 },
 {
 "Index" : 70,
 "Height" : 67.36763,
 "Weight" : 124.7299
 },
 {
 "Index" : 71,
 "Height" : 65.27033,
 "Weight" : 129.3137
 },
 {
 "Index" : 72,
 "Height" : 70.84278,
 "Weight" : 134.0175
 },
 {
 "Index" : 75,
 "Height" : 68.2452,
 "Weight" : 128.5214
 },
 {
 "Index" : 77,
 "Height" : 68.36275,
 "Weight" : 138.6036
 },
 {
 "Index" : 93,
 "Height" : 71.48752,
 "Weight" : 140.6104
 },
 {
 "Index" : 94,
 "Height" : 69.20477,
 "Weight" : 112.1413
 },
 {
 "Index" : 96,
 "Height" : 70.55703,
 "Weight" : 131.8001
 },
 {
 "Index" : 97,
 "Height" : 66.28644,
 "Weight" : 120.0285
 },
 {
 "Index" : 98,
 "Height" : 63.42577,
 "Weight" : 123.0972
 },
 {
 "Index" : 101,
 "Height" : 64.87434,
 "Weight" : 102.0927
 },
 {
 "Index" : 104,
 "Height" : 65.61073,
 "Weight" : 98.64133
 },
 {
 "Index" : 106,
 "Height" : 68.0212,
 "Weight" : 123.4917
 },
 {
 "Index" : 107,
 "Height" : 67.66193,
 "Weight" : 123.048
 },
 {
 "Index" : 109,
 "Height" : 69.43706,
 "Weight" : 128.417
 },
 {
 "Index" : 111,
 "Height" : 67.72277,
 "Weight" : 122.0562
 },
 {
 "Index" : 114,
 "Height" : 65.94588,
 "Weight" : 111.8955
 },
 {
 "Index" : 116,
 "Height" : 68.61129,
 "Weight" : 128.5547
 },
 {
 "Index" : 118,
 "Height" : 69.76212,
 "Weight" : 136.0632
 },
 {
 "Index" : 121,

<iframe id="frame-vegas-1e5f442b-44eb-4f3a-9547-368baae3885f" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-1e5f442b-44eb-4f3a-9547-368baae3885f'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 400.0,
 "height" : 100.0,
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "Index",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "Weight",
 "type" : "quantitative"
 },
 "color" : {
 "field" : "Cylinders",
 "type" : "nominal"
 }
 },
 "description" : "Person Weights",
 "data" : {
 "values" : [
 {
 "Index" : 1,
 "Height" : 65.78331,
 "Weight" : 112.9925
 },
 {
 "Index" : 4,
 "Height" : 68.2166,
 "Weight" : 142.3354
 },
 {
 "Index" : 9,
 "Height" : 67.90265,
 "Weight" : 112.3723
 },
 {
 "Index" : 11,
 "Height" : 66.48769,
 "Weight" : 127.4516
 },
 {
 "Index" : 12,
 "Height" : 67.62333,
 "Weight" : 114.143
 },
 {
 "Index" : 14,
 "Height" : 67.11656,
 "Weight" : 122.4618
 },
 {
 "Index" : 21,
 "Height" : 67.83379,
 "Weight" : 141.2807
 },
 {
 "Index" : 22,
 "Height" : 68.87881,
 "Weight" : 143.5392
 },
 {
 "Index" : 25,
 "Height" : 67.62804,
 "Weight" : 141.8501
 },
 {
 "Index" : 26,
 "Height" : 67.20864,
 "Weight" : 129.7244
 },
 {
 "Index" : 28,
 "Height" : 67.49434,
 "Weight" : 131.5502
 },
 {
 "Index" : 29,
 "Height" : 66.53401,
 "Weight" : 108.3324
 },
 {
 "Index" : 30,
 "Height" : 65.44098,
 "Weight" : 113.8922
 },
 {
 "Index" : 32,
 "Height" : 65.8132,
 "Weight" : 120.7536
 },
 {
 "Index" : 34,
 "Height" : 70.59505,
 "Weight" : 136.2225
 },
 {
 "Index" : 35,
 "Height" : 71.80484,
 "Weight" : 140.1015
 },
 {
 "Index" : 36,
 "Height" : 69.20613,
 "Weight" : 128.7487
 },
 {
 "Index" : 37,
 "Height" : 66.80368,
 "Weight" : 141.7994
 },
 {
 "Index" : 40,
 "Height" : 64.04535,
 "Weight" : 106.7115
 },
 {
 "Index" : 41,
 "Height" : 68.57463,
 "Weight" : 124.3598
 },
 {
 "Index" : 42,
 "Height" : 65.18357,
 "Weight" : 124.8591
 },
 {
 "Index" : 43,
 "Height" : 69.65814,
 "Weight" : 139.6711
 },
 {
 "Index" : 46,
 "Height" : 68.67249,
 "Weight" : 128.7639
 },
 {
 "Index" : 53,
 "Height" : 70.26939,
 "Weight" : 125.4839
 },
 {
 "Index" : 54,
 "Height" : 69.10344,
 "Weight" : 115.7084
 },
 {
 "Index" : 56,
 "Height" : 70.18447,
 "Weight" : 147.8926
 },
 {
 "Index" : 59,
 "Height" : 66.36418,
 "Weight" : 119.3701
 },
 {
 "Index" : 61,
 "Height" : 66.50418,
 "Weight" : 128.7325
 },
 {
 "Index" : 66,
 "Height" : 68.21951,
 "Weight" : 109.6113
 },
 {
 "Index" : 69,
 "Height" : 67.21568,
 "Weight" : 103.5275
 },
 {
 "Index" : 71,
 "Height" : 65.27033,
 "Weight" : 129.3137
 },
 {
 "Index" : 72,
 "Height" : 70.84278,
 "Weight" : 134.0175
 },
 {
 "Index" : 73,
 "Height" : 69.92442,
 "Weight" : 140.3969
 },
 {
 "Index" : 75,
 "Height" : 68.2452,
 "Weight" : 128.5214
 },
 {
 "Index" : 76,
 "Height" : 66.35708,
 "Weight" : 120.2991
 },
 {
 "Index" : 79,
 "Height" : 69.71947,
 "Weight" : 115.6233
 },
 {
 "Index" : 84,
 "Height" : 66.27848,
 "Weight" : 128.9418
 },
 {
 "Index" : 85,
 "Height" : 69.20198,
 "Weight" : 129.1013
 },
 {
 "Index" : 87,
 "Height" : 67.36436,
 "Weight" : 140.8901
 },
 {
 "Index" : 88,
 "Height" : 70.09297,
 "Weight" : 131.5916
 },
 {
 "Index" : 93,
 "Height" : 71.48752,
 "Weight" : 140.6104
 },
 {
 "Index" : 96,
 "Height" : 70.55703,
 "Weight" : 131.8001
 },
 {
 "Index" : 98,
 "Height" : 63.42577,
 "Weight" : 123.0972
 },
 {
 "Index" : 99,
 "Height" : 66.76711,
 "Weight" : 128.1432
 },
 {
 "Index" : 100,
 "Height" : 68.88741,
 "Weight" : 115.4759
 },
 {
 "Index" : 101,
 "Heigh

### Stop application

In [19]:
println("Stop Spark session")
spark.stop()
//if you look in Spark Master UI, no application will be running after stop

Stop Spark session
